In [48]:
import numpy as np
from PIL import Image,ImageDraw
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from ipywidgets import  interactive
from glob import glob
import numpy.typing as npt

In [49]:
def split_shares(img:npt.NDArray):
    """
    img msk s1 s2
    0   0   10 10 
    0   1   01 01
    1   0   10 01
    1   1   01 10
    """
    H, W = img.shape
    s1, s2 = np.zeros((2, H, W, 2))
    mask = np.random.rand(H, W) > 0.5
    
    s1[(img == 1) & (mask == 0)] = [1, 0]
    s2[(img == 1) & (mask == 0)] = [1, 0]
    
    s1[(img == 1) & (mask == 1)] = [0, 1]
    s2[(img == 1) & (mask == 1)] = [0, 1]
    
    s1[(img == 0) & (mask == 0)] = [1, 0]
    s2[(img == 0) & (mask == 0)] = [0, 1]
    
    s1[(img == 0) & (mask == 1)] = [0, 1]
    s2[(img == 0) & (mask == 1)] = [1, 0]
    
    s1 = s1.reshape(H, 2*W).astype(bool)
    s2 = s2.reshape(H, 2*W).astype(bool)
    return s1, s2
def join_shares(x, y):

    return x == y


In [50]:
def hideText(img=None,output_path1='out1.png',output_path2='out2.png'):
    if img is None:
        blank_img = np.zeros((100,100),dtype=  np.uint8)
        img = Image.fromarray(blank_img)
        draw = ImageDraw.Draw(img)
        draw.text((0,0),"Sample text",255)
        img = np.array(img)
        img[img==255] = 1
    else:
        img = Image.open(img).convert('L')
        img = np.array(img)
        img[img==255] = 1

    output_img1,output_img2 = split_shares(img)
    _,(ax1,ax2) = plt.subplots(1,2,figsize=(10,10))
    ax1.imshow(output_img1,cmap='gray')
    ax2.imshow(output_img2,cmap='gray')
    plt.imsave(output_path1,output_img1,cmap='gray')
    plt.imsave(output_path2,output_img2,cmap='gray')
    plt.show()
    

In [51]:
def generate_image_with_text(text:str="Sample Text",path:str="sample.png",imgSize:int=100):
    blank_img = np.zeros((imgSize,imgSize),dtype= np.uint8)
    img = Image.fromarray(blank_img)
    draw = ImageDraw.Draw(img)
    draw.text((0,0),text,255)
    img = np.array(img)
    img[img==255] = 1
    plt.imsave(path,img,cmap='binary')
    
    plt.imshow(img,cmap='gray')

In [52]:
def decrypt_and_show(img_path1:str='out1.png',img_path2:str='out2.png'):
    s1 = Image.open(img_path1).convert('L')
    s1= np.array(s1)
    s2 = Image.open(img_path2).convert('L')
    s2 = np.array(s2)
    s1[s1==255]=1
    s2[s2==255]=1

    z = join_shares(s1, s2)
    gs = gridspec.GridSpec(2, 2)
    plt.figure(figsize=(12,12))
    ax = plt.subplot(gs[0, 0]) # row 0, col 0
    plt.imshow(s1,cmap='gray')
    ax = plt.subplot(gs[0, 1]) # row 0, col 1
    plt.imshow(s2,cmap='gray')
    ax = plt.subplot(gs[1, :]) # row 1, span all columns
    plt.imshow(z,cmap='gray')

# Sekcja INFO

## Autor
Michał Kwarta 145192 L2


## Opis algorytmu
Dzielenie obrazka odbywa się w następujący sposób:
Najpierw generowane są dwie macierze wypełnione zerami o rozmiarze wysokość_obrazka X szerokość_obrazka X 2.
Później generuję maskę typu boolean 1 albo 0 z prawdopodobieństwem 50%.
Póżniej macierzy(nazwijmy je s1 i s2) z zerami wstawiam liczby w następujący sposób:

| img | msk |  s1   |   s2  |
|---- | ----| ----- | ----- |
|  0  |  0  | [1,0] | [1,0] |  
|  0  |  1  | [0,1] | [0,1] |
|  1  |  0  | [1,0] | [0,1] |
|  1  |  1  | [0,1] | [1,0] |

A następnie używam metody reshape z pakietu numpy żeby przekształcić je do rozmiaru wysokość obrazka X 2* szerokość obrazka

Podczas łączenia obrazków posługuję się następującą funkcją, a więc, jeżeli s1==s2 to biały, w przeciwnym przypadku czarny.
| s1 | s2 | z |
|----|--- |---|
|0   |0   |1  |
|0   |1   |0  |
|1   |0   |0  |
|1   |1   |1  |

## Specyfikacja założeń
- Dane wejściowe i wyjściowe - Dowolny czarno biały obrazek(koniecznie skrajna biel i skrajna czerń), chociaż program oferuje możliwość generowania obrazka o rozmiarze (100,100) do (1000,1000) z zadanym tekstem. Dane wyjściowe to obrazki w bieżącym katalogu.
- rozmiar danych - Nieistotny, w przypadku użycia funkcji do generowania obrazka, zadany tekst nie powinien być na tyle duży aby się nie zmieścił się na obrazie( chociaż rozmiar obrazka ustala użytkownik)
- Format danych - czarno biały obrazek png
- Ograniczenia - Raczej brak
- Środowisko - python3 z paczkami do obsługi notebooków jupytera wraz z numpy,matplotlib,PIL 

## Pseudo GUI/Strefa klikalna

Generator obrazka z zadanym tekstem do testów

In [53]:
interactive(generate_image_with_text,text="Sample Text",path="sample.png",imgSize=(100,1000))

interactive(children=(Text(value='Sample Text', description='text'), Text(value='sample.png', description='pat…

Podział obrazka 

In [54]:
interactive(hideText,img=glob("*.png"))

interactive(children=(Dropdown(description='img', options=('sample.png', 'out1.png', 'out2.png', 'kwadrat.png'…

Odkrywanie wiadomości


In [55]:
interactive(decrypt_and_show,img_path1=glob("*.png"),img_path2=glob("*.png"))

interactive(children=(Dropdown(description='img_path1', index=1, options=('sample.png', 'out1.png', 'out2.png'…